# **CONSOLE**

Use below cell as console.

In [0]:
# Move all csv files into the folder movements.
# !bash -c 'mv *.csv movements'

# Delete files generated in a previous execution.
!rm "Accuracy.png"
!rm "Loss.png"
!rm "history.csv"
!rm "history.json"
!rm "model.h5"
!rm "model.json"
!rm "model_plot.png"
!rm "results.txt"
!rm "model_full.h5"
!rm "model.tflite"

# **Importing Libraries**

First, let's load all necessary libraries into the notebook.

In [0]:
import pandas as pd
import numpy as np
import glob
import time
import matplotlib.pyplot as plt
import keras
from keras import Model
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.layers import LSTM, Dense, Dropout, Input, Activation, Masking, Conv1D, Flatten

# **Parameters**

Here, we are going to group the parameters of the notebook, so we can change them easily.

In [0]:
# Path and extension of dataset files.
dataset_path = "movements/*.csv"

# Apply or not EWMA to smooth curves.
ewma = True

# Apply or not normalization to set data in range .
norm = False

# Value of beta in EWMA.
ewma_b = 0.3

# learning rate
adam_learning_rate = 0.0001

# Columns to drop. Options: ["magn_field_x", "magn_field_y", "magn_field_z",  "accelerometer_x", "accelerometer_y", "accelerometer_z", "gravity_x", "gravity_y", "gravity_z", "gyros_x", "gyros_y", "gyros_z", "lin_accel_x", "lin_accel_y", "lin_accel_z", "game_rot_vec_x", "game_rot_vec_y", "game_rot_vec_z"]
drop = []

# Which normalization method use: [std_mean, min_max]
chosen_norm = "min_max"

# Which model train. Options: [ann, rnn, cnn]
chosen_model = "cnn"

# Parameters of Adam optimizer:
adam_beta_1 = 0.9
adam_beta_2 = 0.999
adam_decay = 0.0001

# Model fit parameters:
model_loss = 'logcosh'
model_validation_split = 0.3
model_batch_size = 40
model_epochs = 4000

# **Loading the dataset**

Now, we are going to load the dataset. This dataset is formed by different movements extracted from American Kenpo Karate's blocking set. Each movement is stored in a csv file and has information from the different sensors of an android device.

In [0]:
# Load files of dataset.
files = glob.glob(dataset_path)

# List for storing elements of the dataset.
X_raw = []
Y_raw = []

# Read files and store each example in list.
for path in files:
    df = pd.read_csv(path, sep=",", decimal=".", header=0)
    df = df.apply(pd.to_numeric)
    df = df.drop(drop, axis=1)
    
    # Convert df into list.
    X_raw.append(df.T.values.tolist())

    # Get class from file name.
    Y_raw.append(path.split("/")[1][0])

Let's print one of the movements:

In [0]:
print(X_raw[2][2])
print(Y_raw[3])

# **Normalization**

As a way to speed up learnig, we are going to normalize the dataset. This will make each attribute to have a mean closer to 0. The normalization is done by attribute and not globally.

In [0]:
if(norm):
  X_norm = []
  for example in X_raw:
    resExample = []
    for attribute in example:
      if chosen_norm == "std_mean":
        resExample.append((attribute - np.mean(example)) / np.std(attribute))
      elif chosen_norm == "min_max":
        resExample.append((attribute - np.min(example)) / (np.max(example) - np.min(example)))
    X_norm.append(resExample.copy())
else:
  X_norm = X_raw

Now we have the dataset normalized.

In [0]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(X_norm[1][2], 'tab:blue')
ax1.set_title("Normalized.")
ax2.plot(X_raw[1][2], 'tab:orange')
ax2.set_title("Raw data.")
plt.show()

# **Smoothing the curves**

The curves genedated by the attributes in each sample are so sharp and noisy. We can smooth this curves using Exponentially Weighted Moving Averages. Let's define a function to smooth this curves.

In [0]:
def applyEWMA(beta, dataset):
    pos = 0
    resX = []
    for example in dataset:
        resExample = []
        for attribute in example:
            resAttribute = [attribute[0]]
            for value in attribute:
                resValue = (beta * value) + (1-beta) * resAttribute[pos]
                resAttribute.append(resValue)
                pos += 1
            resAttribute.pop(0)
            resExample.append(resAttribute.copy())
            resAttribute.clear()
            pos = 0
        resX.append(resExample.copy())
        resExample.clear()
    return resX

Now, we can use this function to apply EWMA over the full dataset.

In [0]:
if(ewma):
    X_smooth = applyEWMA(ewma_b, X_norm)

Let's take a look of how we have smoothen our curves.

In [0]:
if(ewma):
  plt.plot(X_norm[1][2], label='Norm')
  plt.plot(X_smooth[1][2], label='Smooth')
  plt.legend()
  plt.show()

By adjusting the value of beta, we can control how smooth the new curves are. Lower values of beta gives smoother curves. If beta is 1, the generated curve is the same as the original.

# **Padding the sequences**

For feeding the neural networks, all sequences must have the same length. Here, the length of the longest sequence is obtained, so all the sequences can be expanded (padded) till reach the length of the longest sequence in the dataset.

In [0]:
maxLength = 0
for example in X_smooth:
  if len(example[0]) > maxLength:
    maxLength = len(example[0])
print("Length of longest sample: " + str(maxLength))

Now, we are going to expand each sample.

In [0]:
# Extend each sample to maximum length
X_expanded = []
for example in X_smooth:
  resExample = pad_sequences(example, padding='post', maxlen=maxLength, dtype='float64')
  X_expanded.append(resExample)

Let's see an example of expanded movement.

In [0]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(X_smooth[1][2], 'tab:blue')
ax1.set_title("Non Expanded.")
ax2.plot(X_expanded[1][2], 'tab:orange')
ax2.set_title("Expanded.")
plt.show()

# **Convert to ndarray**

Right now, we have a list of examples, where each one is a ndarray containing 18 attributes. Each attribute has the length of the previously calculated max length. We should turn that list in an ndarray, resulting in a 3-dimensional ndarray. The output Y should be formed by one-hot vectors, so we will use the function to_categorical to turn each output in a one-hot vector.

In [0]:
# Convert into Array
X = np.asarray(X_expanded)
print(X.shape)
Y = np.asarray(Y_raw)
print(Y.shape)
Y = to_categorical(Y)
print(Y.shape)

As input dataset for trainig, we are going to use the smoothen and normalized dataset X_norm. Let's get its dimensions.

In [0]:
# Get dimensions
num_examples, num_attributes, num_values = X.shape
print(X.shape)
_, num_classes = Y.shape
print("Num. Examples: " + str(num_examples))
print("Num. Attributes: " + str(num_attributes))
print("Num. Values: " + str(num_values))
print("Num. Classes: " + str(num_classes))


# **Create the model**

Now we are going to create three models with the following characteristics: 

*   The first model is using a Conv1D layer. After that, dropout is applied for preventing overfitting, and the data is flattened into a one-dimensional array. Then, that array is passed to a dense layer with 6 units (number of classes), and softmax is applied.
*   The second model is using a Dense layer. After that, dropout is applied for preventing overfitting. Then, the output of the dense layer is passed to another dense layer with 6 units (number of classes), and softmax is applied.
*   The third model is using a LSTM layer. After that, dropout is applied for preventing overfitting, and the data is flattened into a one-dimensional array. Then, that array is passed to a dense layer with 6 units (number of classes), and softmax is applied.

In the three cases below, the value for dropout is 0.5. Those are very simple models, but in the training set used with a good configuration of parameters, achieved accuracies over 0.9 for the training set and over 0.8 in the test set. LSTM is the model that got the better results with 1.0 of accuracy over the training set and 0.94 over the test set.

In [0]:
# Build CNN model.
def cnn_model(input_shape):
  
    input = Input(shape=input_shape, dtype='float32')
    i = Conv1D(filters=32, kernel_size=5, activation='relu')(input)
    i = Dropout(0.5)(i)
    i = Flatten()(i)

    i = Dense(num_classes)(i)
    i = Activation('softmax')(i)

    model = Model(inputs=input, outputs=i)

    return model

In [0]:
# Build ANN model.
def ann_model(input_shape):
  
    input = Input(shape=input_shape, dtype='float32')
    i = Flatten()(input)

    i = Dense(512)(i)
    i = Dropout(0.5)(i)
    i = Dense(num_classes)(i)
    i = Activation('softmax')(i)

    model = Model(inputs=input, outputs=i)

    return model

In [0]:
# Build RNN model.
def rnn_model(input_shape):

    input = Input(shape=input_shape, dtype='float32')
    #i = Masking(mask_value=0.0)(input)
    i = LSTM(56, return_sequences=True)(input)
    i = Dropout(0.5)(i)
    i = Flatten()(i)

    i = Dense(num_classes)(i)
    i = Activation('softmax')(i)

    model = Model(inputs=input, outputs=i)

    return model

Let's create the model and print the summary.

In [0]:
if chosen_model == "rnn":
  model = rnn_model(input_shape=(num_attributes, num_values))
elif chosen_model == "cnn":
  model = cnn_model(input_shape=(num_attributes, num_values))
elif chosen_model == "ann":
  model = ann_model(input_shape=(num_attributes, num_values))
  
model.summary()

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

# **Training the model**

Now let's train the model using an Adam optimizer.


In [0]:
# Instantiate optimizer and compile selected model.
opt = Adam(lr=adam_learning_rate, beta_1=adam_beta_1, beta_2=adam_beta_2, decay=adam_decay)
model.compile(loss=model_loss, optimizer=opt, metrics=["accuracy"])

# Get timestamp.
start_time = time.time()

# Start training. Set verbose to 1 or 2 to print progress.
history = model.fit(X, Y, validation_split=model_validation_split, shuffle=True, batch_size=model_batch_size, epochs=model_epochs, verbose=0)

# Print accuracies and time.
print("Train acc.: %s" % history.history['accuracy'][-1])
print("Test acc.: %s" % history.history['val_accuracy'][-1])
print("Time: %s" % (time.time() - start_time))

# Save accuracies and time into a file.
f = open("results.txt", "w")
f.write("Train acc.: " + str(history.history['accuracy'][-1]) + "\n")
f.write("Test acc.: " + str(history.history['val_accuracy'][-1]) + "\n")
f.write("Time: " + str(time.time() - start_time) + "\n")
f.close()

# **Visualizing results**

Now that we have trained our model, let's take a look to the evolution of loss and accuracy in training and validation set.

In [0]:
# Show accuracy in last epoch
print("Train acc:" + str(history.history['accuracy'][len(history.history['accuracy'])-1]))
print("Test acc:" + str(history.history['val_accuracy'][len(history.history['val_accuracy'])-1]))

# Show data saved in history,
print(history.history.keys())

# Show accuracy and save chart into file.
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('Accuracy.png')
plt.show()

# Show loss and save chart into file.
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('Loss.png')
plt.show()

# **Saving model and weights**

Let's now save the model and the weights into a file.

In [0]:

# Save model to JSON file.
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
# Save weights to HDF5 file.
model.save_weights("model.h5")

# Convert history into pandas df.
hist_df = pd.DataFrame(history.history)
# Save history into JSON file.
hist_json_file = 'history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)
# Save history into CSV file.
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
    
print("Saved model to disk")

# Play a sound indicating that the training has finished.
import IPython.display as display
display.Audio(url="https://static.sfdict.com/audio/C07/C0702600.mp3", autoplay=True)


# **Loading existing model**
An existing model can be loaded for testing over the dataset, continue training, or saving with other formats.

In [0]:
from keras.models import model_from_json
from keras.models import load_model

# Clear previous Keras session.
keras.backend.clear_session()

# Load model from json file.
with open('model.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights from H5DF file.
model.load_weights('model.h5')

# If models and weights are both inside a H5DF file, load with this.
# load_model("model_full.h5")

# Print summary of the model.
model.summary()

# **Convert the model into tflite**
Convert the model into tflite so it can be used in android.

In [0]:
import tensorflow as tf
from keras.models import model_from_json

keras.backend.clear_session()

# Save model and weights into a single H5DF file.
model.save('model_full.h5')

# Load the model using tf.keras.
model = tf.keras.models.load_model('model_full.h5', compile=False)

# Converting a tf.Keras model to a TensorFlow Lite model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TF Lite model.
with tf.io.gfile.GFile('model.tflite', 'wb') as f:
  f.write(tflite_model)